# Ajuste Fino Supervisionado com SFTTrainer

Este caderno demonstra como ajustar o modelo `HuggingFaceTB/SmolLM2-135M` usando o `SFTTrainer` do móduli `trl`. As células do caderno são executadas e vão ajustar finamente o modelo. Você pode selecionar sua dificuldade experimentando diferentes conjuntos de dados.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercício: Ajuste fino do SmolLM2 com SFTTrainer</h2>
    <p>Pegue um conjunto de dados do hub do Hugging Face e faça um ajuste fino em um modelo. </p> 
    <p><b>Níveis de Dificuldade</b></p>
    <p>🐢 Use o conjunto de dados `HuggingFaceTB/smoltalk`.</p>
    <p>🐕 Experimente o conjunto de dados `bigcode/the-stack-smol` e ajuste um modelo de geração de código no subconjunto específico `data/python`.</p>
    <p>🦁 Selecione um conjunto de dados relacionado a um caso de uso do mundo real que seja do seu interesse.</p>
</div>

In [ ]:
# Instale os requisitos no Google Colab
# !pip install transformers datasets trl huggingface_hub

# Autentique ao Hugging Face

from huggingface_hub import login
login()

# Por conveniência, você pode criar uma variável de ambiente que contém seu token do hub como HF_TOKEN

In [ ]:
# Importe módulos necessários
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Carregue o modelo e o tokenizador
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Configure o formato de bate-papo
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Defina nosso nome para o ajuste fino a ser salvo e/ou carregado em um local
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Gerar com o modelo base

Aqui vamos experimentar o modelo base que não tem um modelo de bate-papo. 

In [ ]:
# Vamos testar o modelo base antes do treinamento
prompt = "Write a haiku about programming"

# Formato com o modelo de bate-papo
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Gerando a resposta
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## Preparação do Conjunto de Dados

Carregaremos um conjunto de dados de amostra e o formataremos para treinamento. O conjunto de dados deve ser estruturado com pares de input-output, em que cada input é um prompt e o output é a resposta esperada do modelo.

**TRL formatará as mensagens de entrada com base nos modelos de bate-papo.** Elas precisam ser representadas como uma lista de dicionários com as chaves: `role` e `content`.

In [ ]:
# Carregue um conjunto de dados de amostra
from datasets import load_dataset

# TODO: Defina seu próprio conjunto de dados e configurações usando os parâmetros do caminho e do nome
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 Se o seu conjunto de dados não estiver em um formato que o TRL possa converter para o modelo de chat, você precisará processá-lo. Consulte o [módulo](../chat_templates.md)

## Configuração do SFTTrainer

O `SFTTrainer` é configurado com vários parâmetros que controlam o processo de treinamento. Eles incluem o número de etapas de treinamento, o tamanho do lote, a taxa de aprendizado e a estratégia de avaliação. Ajuste esses parâmetros com base em seus requisitos específicos e recursos computacionais.

In [ ]:
# Configure o SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Inicie o SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 alinhe os parâmetros do SFTTrainer com o conjunto de dados escolhido. Por exemplo, se estiver usando o conjunto de dados `bigcode/the-stack-smol`, será necessário escolher a coluna `content`

## Treinando o modelo

Com o trainer configurado, podemos prosseguir com o treinamento do modelo. O processo de treinamento envolverá a iteração do conjunto de dados, o cálculo da perda e a atualização dos parâmetros do modelo para minimizar essa perda.

In [ ]:
# Treine o modelo
trainer.train()

# Salve o model
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercício bônus: Gerar com o modelo ajustado</h2>
    <p>🐕 Use o ajuste fino para modelar e gerar uma resposta, assim como no exemplo básico.</p>
</div>

In [ ]:
# Teste o modelo ajustado no mesmo prompt

# Vamos testar o modelo básico antes do treinamento
prompt = "Write a haiku about programming"

# Formatar com o modelo
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Gerar resposta
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use o ajuste fino para modelar e gerar uma resposta, assim como no exemplo básico.

## 💐 Você conseguiu!

Este caderno fornece um guia passo-a-passo para o ajuste fino do modelo `HuggingFaceTB/SmolLM2-135M` usando o `SFTTrainer`. Seguindo essas etapas, você pode adaptar o modelo para executar tarefas específicas com mais eficiência. Se quiser continuar trabalhando neste curso, aqui estão as etapas que você pode experimentar:

- Experimente este caderno com uma dificuldade maior
- Revisar o PR de um colega
- Melhorar o material do curso por meio de uma Issue ou PR.